# Section 3: `pandas`, `pickles`, and Lambda functions

GSIs: Simon Greenhill & Hikari Murayama

Fall 2022

<font size='1'> *Notebook originally created by Jaecheol Lee

### Plan for Today

1. `pandas`
2. `pickles`
3. $\lambda$ calculation
$$
\hat{\lambda}(\overrightarrow{\underset{\cdot}{s}}) =\frac{1}{\pi h^{2}} \times \sum_{j=1}^{N} \mathbf{1}\left[ dist(\overrightarrow{\underset{\cdot}{s}}, \overrightarrow{\underset{\cdot}{s_{inf\_site\; j}}})< h \right]
$$
4. K function
$$\hat{K}(h) = \frac{1}{z\lambda}(\sum_{i=1}^z \sum_{j \neq i} \mathbf{1}\{D(\overrightarrow{\underset{\cdot}{s_i}}, \overrightarrow{\underset{\cdot}{s_j}}) < h\})$$

## Import Packages

In [1]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## 1. `pandas`

### We will introduce pandas. Why pandas?
1. The most native and powerful Python library to handle real-world tabular data (as in an Excel spreadsheet).


2. Efficiency: Many of the low-level algorithms have been extensively optimized. Handles large data frames.


3. Mostly intuitive syntax, despite being a bit clunky at times.


### 1-1. Loading and Inspecing Data

In [14]:
# When you load crime.csv in the same 
df = pd.read_csv('crime.csv')
type(df)

pandas.core.frame.DataFrame

In [16]:
# We can also import web files
# iris.csv from https://github.com/mwaskom/seaborn-data/
# Note that the url below is a bit different
iris = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv')
type(iris)
iris

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [17]:
# Let's see the data
df 

,incident_id,type,lat,lon
0,2020-00049667,ROBBERY,37.865137,-122.260614
1,2020-00049522,ASSAULT,37.867621,-122.260844
2,2020-00049500,BURGLARY AUTO,37.869658,-122.267919
3,2020-00049505,ROBBERY,37.874252,-122.267559
4,2020-00049352,THEFT,NaN,NaN


In [42]:
# Check the column names using .columns method
df.columns

Index(['incident_id', 'type', 'lat', 'lon'], dtype='object')

In [43]:
# What is the dimension (rows #, columns #) of df? 
df.shape

(5, 4)

In [44]:
# Summarize statistics of a pandas.dataframe using .describe method
df.describe()

,lat,lon
count,4.000000,4.000000
mean,37.869167,-122.264234
std,0.003861,0.004051
min,37.865137,-122.267919
25%,37.867000,-122.267649
50%,37.868640,-122.264201
75%,37.870807,-122.260786
max,37.874252,-122.260614


In [ ]:
# Say I want to add a new year column
# We can now add it to our dataframe as a new column
df['year'] = [2020,2020,2020,2020,2020]
df

In [36]:
# We can drop a column
# axis 1: is column (axis =0 would be a row)
# inplace = True is needed for assignment
df.drop(['year'], axis = 1, inplace = True)
df

In [38]:
# Now let's drop the first row
df.drop([0], axis = 0, inplace = True)
df

In [40]:
# When you load crime.csv in the same 
df = pd.read_csv('crime.csv')
df

<div class="alert alert-block alert-info">
<b>Your turn!</b> 

1. What is the third column in our dataframe?
2. Create a new column where you have the incident id without the year prefix
    
</div>

In [ ]:
# Your code here

### 1-2. `pandas` indexing

1. “Pandas is a best friend to a data scientist, and index is the invisible soul behind pandas.”
2. Indexing is crucial to performing many key operations, including selecting, grouping and merging data.
3. The idea: Every observation should be uniquely identified by one or several variables (can be integer, string or datetime object, etc.). Every row and column has a ‘name’: row: df.index and column: df.columns

Two preferred approaches

- indexing by label: `df.loc[]`

- indexing by position: `df.iloc[]`

Also common (indexing as if indexing a dictionary):

- `df[‘type’]` gives a pandas Series

- `df[[‘type’]]` or `df[[‘lat’, ‘lon’]]` gives a `pandas` DataFrame

In [3]:
# Before setting the index, the basic syntax is df.loc[row_number, col_label]
# Only the first row
df.loc[0, :]

incident_id    2020-00049667
type                 ROBBERY
lat                37.865137
lon              -122.260614
Name: 0, dtype: object

In [4]:
# The first row, 'type' column
df.loc[0, 'type']

'ROBBERY'

In [5]:
# Only 'type' column
df.loc[:, 'type']

0          ROBBERY
1          ASSAULT
2    BURGLARY AUTO
3          ROBBERY
4            THEFT
Name: type, dtype: object

In [6]:
# After setting the index, the basic syntax is df.loc[row_label, col_label] 
df.set_index('incident_id', inplace=True)

df

,lat
incident_id,
2020-00049500,37.869658


In [ ]:
# Depending on how you index, the object type will be different 

df.loc['2020-00049500', :] # a Series 
df.loc['2020-00049500', 'lat'] # a number 
df.loc[['2020-00049500'], 'lat'] # a Series 
df.loc[['2020-00049500'], ['lat']] # a DataFrame 

In [7]:
# loc takes many different input types such as boolean sequence
df.loc[[True, True, True, False, False], :] 
df.loc[df['type'] == 'ROBBERY', :]

,type,lat,lon
incident_id,,,
2020-00049667,ROBBERY,37.865137,-122.260614
2020-00049505,ROBBERY,37.874252,-122.267559


In [8]:
# You can index a data by using numbers
# The basic syntax is df.iloc[row_pos, col_pos]
df.iloc[0, :]
df.iloc[[0, 1], [2]]
# You cannot mix label based and position based indexing 
# e.g., df.iloc[0, 'lat']

,lon
incident_id,
2020-00049667,-122.260614
2020-00049522,-122.260844


In [9]:
# The simplest way? When you want to use only some columns:
# One column
df['type']

## Two columns: list of column names
df[['type', 'lat']]

,type,lat
incident_id,,
2020-00049667,ROBBERY,37.865137
2020-00049522,ASSAULT,37.867621
2020-00049500,BURGLARY AUTO,37.869658
2020-00049505,ROBBERY,37.874252
2020-00049352,THEFT,NaN


In [10]:
df

,type,lat,lon
incident_id,,,
2020-00049667,ROBBERY,37.865137,-122.260614
2020-00049522,ASSAULT,37.867621,-122.260844
2020-00049500,BURGLARY AUTO,37.869658,-122.267919
2020-00049505,ROBBERY,37.874252,-122.267559
2020-00049352,THEFT,NaN,NaN


In [12]:
df.index

Index(['2020-00049667', '2020-00049522', '2020-00049500', '2020-00049505',
       '2020-00049352'],
      dtype='object', name='incident_id')

<div class="alert alert-block alert-info">
<b>Your turn!</b> 

1. Filter your dataframe to only include those that are BURGLARY AUTO

2. Return the longitude of the crime with incident id 2020-00049352
 
    
</div>

In [ ]:
# Your code here

#### Missing Data

In the real world, you'll encounter dataframes that contain missing information. You'll need to know how to deal with these.

In [ ]:
# Drop the missing data
df.dropna(how = 'any') # inplace argument is False by default

In [ ]:
# Which are None elements?
pd.isna(df) 

In [ ]:
df['lat'].isna() 

In [ ]:
df['lat'].notna()

#### String Manipulations

Perhaps you know that there is valuable inormation in a variable you already have. The same way we manipulate objects that aren't in a dataframe, you can do for variables in a dataframe.

In [ ]:
df['type'].str.lower()
df['incident_id0'] = df.index.str.split('-').str[0] 
df['incident_id1'] = df.index.str.split('-').str[1] 
df['type'].str.match('THEFT')
df

#### Grouping and Summarizing

These are good to know when you want some highlevel information on your data

In [ ]:
# import numpy as np
grouped = df.groupby('type')

# the following two are equivalent 
grouped.mean() 
grouped.aggregate(np.mean)

#### Merging datasets
There are instances where your data might exist in two different dataframes, but you have a unique identifier between the two.  You can use this to merge two dataframes together.

![merge](https://miro.medium.com/max/1400/1*7a9Tzz6z8H6Cy7PRhHiJGQ.png)


In [ ]:
# Theoretically you could do this with another crime dataset like its outlined here
# df_to_join = pd.read_csv('another_crime_dataset.csv') 
# pd.merge(df, df_to_join, on='type', how='left') 
# pd.merge(df, df_to_join, on='type', how='right') 
# pd.merge(df, df_to_join, on='type', how='inner') 
# pd.merge(df, df_to_join, on='type', how='outer')

<div class="alert alert-block alert-info">
<b>Your turn!</b> 

Say you have two tables A and B
    
Table A:

| id | population_humans |
| --- | --- | 
| 1 | 100|
| 2 | 100|
| 4 | 50|
| 5 | 3|

Table B:

| id | population_dogs |
| --- | --- | 
| 1 | 120 |
| 2 | 3000|
| 3 | 5|
    


If A is your left datframe, and B is your right dataframe, how many rows would you have when you
1. Do a left join
2. Do a right join
3. Do an inner join
4. Do an outer join
    
</div>


    

In [ ]:
# You answer here

## 2. `pickle`

You'll see in Lab 3 that you need to open a pickle file. We'll briefly discuss what it is, how to open it, and how to save it.

In [100]:
import pickle
# the with statement opens the file, and then (silently) <<< closes it >>>
# 'rb' stands for 'r'eading mode and 'b'inary mode (pickle saves objects into a byte stream)
# f is called a file handle. Think of it as a pointer to the file
# it doesn't have to be f, of course, name it whatever you like
# there is a convention to name it f, though
with open('Lab2_houses.p', 'rb') as f:

    # we load the file by calling pickle.load()
    # and then assign it to d, which is a short name that we gave to this dict
    d = pickle.load(f)
    # here (at the end of the with statement), python silently <<< closed the file >>>

# f = open('Lab2_houses.p', 'rb')
# d = pickle.load(f)

##### What does it mean by 'closing the file'?

In [101]:
# This is for checking the list of files which are open at the moment
# You don't need to run this, but you can install psutil if you want to try this. 
import psutil
proc = psutil.Process()
proc.open_files()

[popenfile(path='/Users/jaecheollee/.ipython/profile_default/history.sqlite', fd=48),
 popenfile(path='/Users/jaecheollee/.ipython/profile_default/history.sqlite', fd=49),
 popenfile(path='/Users/jaecheollee/opt/anaconda3/envs/pp275/lib/python3.8/site-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSans.ttf', fd=62)]

In [102]:
# The above cell is the same as below:
test = open('Lab2_houses.p', 'rb')
d2 = pickle.load(test)

In [103]:
# Check the list of files
proc.open_files()

[popenfile(path='/Users/jaecheollee/.ipython/profile_default/history.sqlite', fd=48),
 popenfile(path='/Users/jaecheollee/.ipython/profile_default/history.sqlite', fd=49),
 popenfile(path='/Users/jaecheollee/opt/anaconda3/envs/pp275/lib/python3.8/site-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSans.ttf', fd=62),
 popenfile(path='/Users/jaecheollee/Desktop/SA/Sections/Section2/Lab2_houses.p', fd=63)]

In [104]:
# Check the list of files after closing object test. 
test.close()
proc.open_files()

[popenfile(path='/Users/jaecheollee/.ipython/profile_default/history.sqlite', fd=48),
 popenfile(path='/Users/jaecheollee/.ipython/profile_default/history.sqlite', fd=49),
 popenfile(path='/Users/jaecheollee/opt/anaconda3/envs/pp275/lib/python3.8/site-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSans.ttf', fd=62)]

##### The reasons we should 'close' files:
(from https://stackoverflow.com/questions/25070854/why-should-i-close-files-in-python)
1. It can slow down your program. Too many things open, and thus more used space in the RAM, will impact performance.
2. For the most part, many changes to files in python do not go into effect until after the file is closed, so if your script edits, leaves open, and reads a file, it won't see the edits.
3. You could, theoretically, run in to limits of how many files you can have open.

**Using `with open` closes the file automatically!**

### 3. Lambda calculation

$$
\hat{\lambda}(\overrightarrow{\underset{\cdot}{s}}) =\frac{1}{\pi h^{2}} \times \sum_{j=1}^{N} \mathbf{1}\left[ dist(\overrightarrow{\underset{\cdot}{s}}, \overrightarrow{\underset{\cdot}{s_{inf\_site\; j}}})< h \right]
$$

In Lab 3, we'll construct an estimate for the rate at which cholera cases arise (the infection rate) at each location on a lattice by using a kernel estimate. This is calcualted using the above equation.

#### 3-1. An example

In [ ]:
# For getting the same result
np.random.seed(123)

# Random events
x_coords = np.random.random(10) * 10
y_coords = np.random.random(10) * 10

In [ ]:
# Plot the x_coords and the y_coords


In [ ]:
# Define a function to calculate the distance between two points
def compute_distance(x0, y0, x1, y1):


In [ ]:
# Calculate the distance between the events and a point x = 3, y = 3


In [ ]:
# Calculate the number of cases where the distance < 2
# Strategy: Get the list of boolean and sum it up!
# Does this work?
distance < 2

# What can we do?

In [ ]:
# Calculate the lambda given a point x = 3, y = 3 and the parameter h = 2


#### Don't be confused with a lambda function! 

1. A lambda function is a small anonymous function (defined without a name).
2. A lambda function can take any number of arguments, but can only have one expression.

##### Why do we use lambda functions? 
https://stackoverflow.com/questions/890128/why-are-python-lambdas-useful

In [ ]:
# lambda is a one-line function:
x = lambda a : a + 10
x(2)

In [ ]:
# Equivalent to the lambda above
def x(a):
    a + 10

x(2)

#### 3-2. With np.meshgrid function and np.ndarray.flatten method

#### Objective: Calculate the lambda for each point in a 10 X 10 grid.

In [ ]:
# How can we make a 10 X 10 grid?
# (The coordinates starts from (1, 1) and ends at (10, 10)
# Use np.meshgrid
x, y = np.meshgrid(range(1, 11), range(1, 11))
# If we draw... 

# How can we get all the coordinates?
x, y = x.flatten(), y.flatten()


In [ ]:
# Plot the events (x_coords, y_coords) and the x, y coordinates in one plot.
# With different markers
plt.plot(x_coords, y_coords, 'ro')
plt.plot(x, y, 'k+')

<div class="alert alert-block alert-info">
<b>Your turn!</b> 

Make a matrix of lambdas for all the locations!
    
Note: There are tons of ways to do this!
</div>

In [ ]:
# Define a function to calculate the lambda given a point
def lambda_function(x, y):


In [ ]:
# Make a matrix of lambdas for all the locations
# utilizing the function above and a double loop
matrix = np.zeros((10, 10))

for i in range(10):
    for j in range(10):


### 4. Understanding K-function

$$\hat{K}(h) = \frac{1}{z\lambda}(\sum_{i=1}^z \sum_{j \neq i} \mathbf{1}\{D(\overrightarrow{\underset{\cdot}{s_i}}, \overrightarrow{\underset{\cdot}{s_j}}) < h\})$$

Now we will compute the distance from each observation to all other observations, rather than the distance from a lattice point to all other observations. (That is why you see $j \neq i $).Note that the K-function is a 'function' of h, the radius parameter.

In [ ]:
# Assume that the lambda parameter is 0.1.
# z is the number of occurrences. 
# Calculate the K for x_coords and y_coords